In [4]:
import pandas as pd
import error_prompts as p
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd

# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()



prompt_name = 'error_class_LLM_nounderover_more_classes'
prompt = p.error_class_LLM_nounderover_more_classes

file = "structured_unstructured_zero_shot_pipeline_3.5-turbo"

DATA_PATH = f'../../Datasets/Evaluations/Regression/Error_Analysis/{file}.csv'
OUTPUT_PATH = f"../../Datasets/Evaluations/Regression/Error_Analysis/error_class_LLM_analysis.csv"

MODEL = "gpt-4-turbo-preview"
TEMP = 0

struct_unstruct = 'structured_analysis'





In [5]:
df = pd.read_csv(DATA_PATH)

df_correct = df[df['error'] == 'correct']
df_correct_samples = df_correct.sample(n=2, random_state=42)

df_error = df[df['error'] == 'error']
df_error_samples = df_error.sample(n=10, random_state=42)

print(df_correct_samples.shape, df_error_samples.shape)
print(df_correct_samples.head())
print(df_error_samples.head())

df_answer = pd.read_csv(OUTPUT_PATH)



(2, 21) (10, 21)
     Unnamed: 0                     name  year  selling_price  km_driven  \
51           51             Tata Hexa XM  2017        1200000      30000   
184         184  Maruti Wagon R VXI Plus  2018         465000      30000   

       fuel seller_type transmission        owner     mileage  ...  \
51   Diesel  Individual       Manual  First Owner   17.6 kmpl  ...   
184  Petrol  Individual       Manual  First Owner  20.51 kmpl  ...   

      max_power               torque seats  prediction  \
51   153.86 bhp  400Nm@ 1750-2500rpm   7.0     1200000   
184   67.04 bhp        90Nm@ 3500rpm   5.0      400000   

                                                prompt prompt_name  \
51   System: Based on the provided attributes of a ...   zero_shot   
184  System: Based on the provided attributes of a ...   zero_shot   

        model_name    error  \
51   gpt-3.5-turbo  correct   
184  gpt-3.5-turbo  correct   

                                   structured_analysis  \
51   

In [6]:
#Structured Analysis 

def execute(df, prompt, df_answer):
    i = 1
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, timeout=100)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, timeout=100)
    examples = ""
    examples += "False Predictions:\n"
    for j in range(len(df_error_samples)):
        examples += f"Task {i}:\n"
        examples += "Prompt: " + df_error_samples.iloc[j]['prompt'] + "\n"
        examples += "Prediction: " + str(df_error_samples.iloc[j]['prediction']) + "\n"
        examples += "Explanation: " + df_error_samples.iloc[j][struct_unstruct] + "\n"
        examples += "Actual Price: " + str(df_error_samples.iloc[j]['selling_price']) + "\n"
        i += 1
    examples += "\nCorrect Predictions (Only as a reference):\n"
    for j in range(len(df_correct_samples)):
        examples += f"Task {i}:\n"
        examples += "Prompt: " + df_correct_samples.iloc[j]['prompt'] + "\n"
        examples += "Prediction: " + str(df_correct_samples.iloc[j]['prediction']) + "\n"
        examples += "Explanation: " + df_correct_samples.iloc[j][struct_unstruct] + "\n"
        examples += "Actual Price: " + str(df_correct_samples.iloc[j]['selling_price']) + "\n"
        i += 1
    chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
    try: 
        result = chain.run(examples = examples, callbacks=[handler])
    except Exception as e:
        print(e)
        print(chain.prompt.format_prompt(examples = examples).to_string())
    handler.langfuse.flush()
    prompt_text = chain.prompt.format_prompt(examples = examples).to_string()
    print(prompt_text)
    print("\n")
    print(result)
    print("\n")
    new_row = {'Prompt_Name': prompt_name, 'Prompt': prompt_text, 'Model': MODEL, 'Database': file, 'Answer': result, 'Context': struct_unstruct}
    df_new_row = pd.DataFrame([new_row])
    df_answer = pd.concat([df_answer, df_new_row])
    return result, df_answer


answer, df_answer = execute(df, prompt, df_answer)
    



In the following I will give you a few price prediction tasks together with a prediction decision, details about the decision and the actual price.
A wrong prediction is a prediction that deviates by more than 20 percent from the actual price. The Prediction was made by an LLM.
Can you please group the wrong decisions into 5 fault categories? Please also indicate how often each one occurs.
There are also some correct decisions (deviation of less than 20 percent) in the examples. Please just use them as a reference and don't categorize them.
Please go deeper with your analysis and don't use Overestimation or Underestimation as categories.
False Predictions:
Task 1:
Prompt: System: Based on the provided attributes of a used car listed below, please predict its selling price in Indian Rupees in the Indian market. The predicted price should be expressed solely as a number followed by the currency "INR".
Ensure that the output contains no additional text or characters beyond this specified 

In [4]:
print(df_answer)

                   Prompt_Name  \
0              error_class_LLM   
1              error_class_LLM   
2              error_class_LLM   
3  error_class_LLM_moreclasses   
4  error_class_LLM_moreclasses   
5  error_class_LLM_moreclasses   
6              error_class_LLM   
0  error_class_LLM_nounderover   

                                              Prompt                Model  \
0  In the following I will give you a few price p...        gpt-3.5-turbo   
1  In the following I will give you a few price p...  gpt-4-turbo-preview   
2  In the following I will give you a few price p...  gpt-4-turbo-preview   
3  In the following I will give you a few price p...  gpt-4-turbo-preview   
4  In the following I will give you a few price p...  gpt-4-turbo-preview   
5  In the following I will give you a few price p...  gpt-4-turbo-preview   
6  In the following I will give you a few price p...  gpt-4-turbo-preview   
0  In the following I will give you a few price p...  gpt-4-turbo-preview   


In [24]:
df_answer.to_csv(OUTPUT_PATH, index=False)